In [1]:
import itertools

import numpy as np
import pandas as pd

from xfirst.atmosphere import us_standard

Welcome to JupyROOT 6.28/04


In [2]:
atm = us_standard()

### Cut due to the zenith angle

The zenith angle of a shower will impact the amount of atmosphere it traverses both until it enters the field of view of a telescope and also the mass overburden at ground. While the maximum height a telescope can observe will depend on the distance to the shower core, the minimum height is always the ground. As for the Pierre Auger Observatory, we define the ground altitude as $h_\mathrm{gr} = 1400$ m.

In [3]:
ground_altitude_km = 1.400 # km

The corresponding mass overburden at ground ($T_\mathrm{gr}$) is:

In [4]:
ground_mass_overburden = atm.get_depth(ground_altitude_km)
ground_mass_overburden

875.5011058303304

For a given zenith angle $\theta$, then, the amount of mass a shower has traversed can be computed by, assuming a linear atmosphere approximation,

$$ T_\mathrm{max} = \frac{T_\mathrm{gr}(h_\mathrm{gr})}{\cos\theta} \:. $$

This expression is implemented by the `get_ground_depth` function below.

In [5]:
def get_ground_depth(theta_shower_rad):
  vertical_depth = atm.get_depth(ground_altitude_km)
  return vertical_depth/np.cos(theta_shower_rad)

Taking the representative zenith angles of 30º, 45º, and 60º we get the following slant depths at the impact point.

In [6]:
theta_range_deg = [30., 45., 60.]

print(f' angle  slant depth')

for theta_deg in theta_range_deg:
  theta_rad = np.radians(theta_deg)
  slant_depth = get_ground_depth(theta_rad)
  print(f'{theta_deg:5}º  {slant_depth:.0f} g/cm²')

 angle  slant depth
 30.0º  1011 g/cm²
 45.0º  1238 g/cm²
 60.0º  1751 g/cm²


### Cut due to core distance

The core distance, on the other hand, determines the maximum height in the atmosphere a telescope can observe. Assume the shower direction specified by the angles $\theta_\mathrm{sh}$ and $\varphi_\mathrm{sh}$, where $\theta_\mathrm{sh}$ is measured with respect to the vertical and $\varphi_\mathrm{sh} = 180º$ means the shower is coming towards the telescope. Then, if the telescope has a vertical field of view of $\theta_\mathrm{fov}$, the maximum altitude a telescope will observe as a function of the distance to the shower core position $r_\mathrm{core}$ is given by

$$h_\mathrm{fov} = r_\mathrm{core} \times \frac{\sin\theta_\mathrm{fov}\sin\theta_\mathrm{sh}}{\cos\theta_\mathrm{fov}\cos\theta_\mathrm{sh} + \sin\theta_\mathrm{fov}\sin\theta_\mathrm{sh}\cos\varphi_\mathrm{sh}} \: .$$

In Auger, the telescope field of view is of 30º.

In [7]:
telescope_fov_deg = 30
telescope_fov_rad = np.radians(telescope_fov_deg)

The $h_\mathrm{fov}$ can be computed using the `get_fov_height` function below.

In [8]:
def get_fov_height(r_core_km, theta_shower_rad, theta_phi_rad):
  sf = np.sin(telescope_fov_rad)
  cf = np.cos(telescope_fov_rad)
  ss = np.sin(theta_shower_rad)
  cs = np.cos(theta_shower_rad)
  den = cf*cs + sf*ss*np.cos(theta_phi_rad)
  return np.inf if (np.abs(den) < 1e-10) else r_core_km*sf*cs/den

The value of $h_\mathrm{fov}$ can be converted into a minimum observable slant depth both as a function of the shower direction and distance to shower core using

$$ T_\mathrm{min} = \frac{T(h_\mathrm{fov} + h_\mathrm{gr})}{\cos\theta_\mathrm{sh}} \: , $$

where $T(h)$ is the vertical mass overburden as a function of the altitude $h$ above sea level. This expression is implemented in the function below.

In [9]:
def get_fov_depth(r_core_km, theta_shower_rad, theta_phi_rad):
  height = ground_altitude_km + get_fov_height(r_core_km, theta_shower_rad, theta_phi_rad)
  vertical_depth = atm.get_depth(height)
  slant_depth = vertical_depth/np.cos(theta_shower_rad)
  return slant_depth

Take the represetative core ranges of 5 km, 10 km, and 20 km and assume the shower has an azimuth angle of 135º.

In [10]:
core_range_km = [5., 10., 20.]

shower_phi_deg = 135
shower_phi_rad = np.radians(shower_phi_deg)

Then we can compute the ranges of slant depths observed by a telescope for different values of $\theta_\mathrm{sh}$ and $r_\mathrm{core}$.

In [11]:
def round(value, precision = 50):
  return precision*(int(value)//precision) + precision*int(np.rint((int(value)%precision)/precision))

# print('theta distance min_depth max_depth')

data = []

for theta_deg in theta_range_deg:
  theta_rad = np.radians(theta_deg)
  max_depth = get_ground_depth(theta_rad)

  print()

  for r_core_km in core_range_km:
    min_depth = get_fov_depth(r_core_km, theta_rad, shower_phi_rad)
    a = round(min_depth, 50)
    b = round(max_depth, 50)
    # print(f'{int(theta_deg):>4}º {r_core_km:>8.1f} {round(min_depth, 50):9} {round(max_depth, 50):9}')

    data.append((theta_deg, r_core_km, a, b))

### Final selection

In [12]:
index = pd.Index([f'{a}{b}' for a, b in itertools.product(['A', 'B', 'C'], range(1, 4))], name = 'cut name')
columns = ['theta_deg', 'r_core_km', 'min_depth', 'max_depth']
pd.DataFrame(data, index = index, columns = columns)

,theta_deg,r_core_km,min_depth,max_depth
cut name,,,,
A1,30.0,5.0,600,1000
A2,30.0,10.0,350,1000
A3,30.0,20.0,100,1000
B1,45.0,5.0,650,1250
B2,45.0,10.0,300,1250
B3,45.0,20.0,50,1250
C1,60.0,5.0,450,1750
C2,60.0,10.0,100,1750
C3,60.0,20.0,0,1750
